<a href="https://colab.research.google.com/github/Metalface-code/Alex-Trimiew-s-Portfolio-/blob/main/Project_1_Final_(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [500]:
import pandas as pd
import numpy as np 
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn import set_config
set_config(display='diagram')

In [501]:
def evaluate_regression(true, pred):
  """Takes true and predicted values (arrays) and prints MAE, MSE, RMSE and R2"""
  mae = mean_absolute_error(true, pred)
  mse = mean_squared_error(true, pred)
  rmse = np.sqrt(mse)
  r2 = r2_score(true, pred)

  print(f'MAE {mae},\n MSE {mse},\n RMSE: {rmse},\n R^2: {r2} ')

In [502]:
fpath = '/content/drive/MyDrive/Coding Dojo/sales_predictions.csv'
df = pd.read_csv(fpath)
df.head(3)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700


In [503]:
ml_df = df.copy()

In [504]:
ml_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [505]:
ml_df.duplicated().sum()

0

In [506]:
ml_df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [507]:
ml_df.dropna(inplace = True)

This week, you will finalize your sales prediction project. The goal of this is to help the retailer understand the properties of products and outlets that play crucial roles in predicting sales.

1) Your first task is to build a linear regression model to predict sales.

Build a linear regression model.
Evaluate the performance of your model based on r^2.
Evaluate the performance of your model based on rmse.

In [508]:
X = ml_df.drop(columns=['Item_Identifier','Outlet_Identifier', 'Item_Outlet_Sales', 'Item_Fat_Content', 'Item_Type', 'Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type', 'Outlet_Type'])
y = ml_df['Item_Outlet_Sales']

In [509]:
ml_df.head(1)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.3,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138


In [510]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
X_train.shape

(3487, 3)

In [511]:
num_selector = make_column_selector(dtype_include='number')
cat_selector = make_column_selector(dtype_include='object')

In [512]:
scaler = StandardScaler()
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [513]:
num_tuple = (scaler, num_selector)
cat_tuple = (encoder, cat_selector)

In [514]:
preprocessor = make_column_transformer(num_tuple, cat_tuple, remainder = 'drop')

In [515]:
dummy_reg = DummyRegressor(strategy='mean')


dummy_pipe = make_pipeline(preprocessor, dummy_reg)

dummy_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f0012824210>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f00128241d0>)])),
                ('dummyregressor', DummyRegressor())])

In [516]:
train_pred = dummy_pipe.predict(X_train)
test_pred = dummy_pipe.predict(X_test)

In [517]:
eval_regression(y_train, train_pred)

MAE 1194.959505418405,
 MSE 2250745.792931671,
 RMSE: 1500.2485770470407,
 R^2: 0.0 


In [518]:
eval_regression(y_test, test_pred)

MAE 1196.67851223271,
 MSE 2221487.683868447,
 RMSE: 1490.4655929837652,
 R^2: -8.864434349242778e-05 


2) Your second task is to build a regression tree model to predict sales.

Build a simple regression tree model.
Compare the performance of your model based on r^2.
Compare the performance of your model based on rmse.  

In [519]:
dec_tree = DecisionTreeRegressor(random_state = 42)
dec_tree.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [520]:
evaluate_regression(y_train, dec_tree.predict(X_train))

MAE 0.0,
 MSE 0.0,
 RMSE: 0.0,
 R^2: 1.0 


3) You now have tried 2 different models on your data set. You need to determine which model to implement.

Overall, which model do you recommend?
Justify your recommendation.

Answer: I would implement the decision tree model becuase the testing seemed to have less overfitting values and made for a better model for predicting the item sales

4) To finalize this project, complete a README in your GitHub repository including:

An overview of the project
2 relevant insights from the data (supported with reporting quality visualizations)
Summary of the model and its evaluation metrics
Final recommendations.